# Experiment

### Imports

In [ ]:
import transform
import numpy as np
from skimage.io import imread
from IPython.display import display, Markdown, Image, HTML
import PIL
from io import BytesIO
from matplotlib import pyplot as plt

### Constants

In [ ]:
DETECTOR_COUNT_MIN = 90
DETECTOR_COUNT_MAX = 720
DETECTOR_COUNT_STEP = 90
DETECTOR_COUNT_DEFAULT = 180
DETECTOR_COUNT_SECOND_EXPERIMENT = 360

ITERATION_COUNT_MIN = 90
ITERATION_COUNT_MAX = 720
ITERATION_COUNT_STEP = 90
ITERATION_COUNT_DEFAULT = 180
ITERATION_COUNT_SECOND_EXPERIMENT = 360

DETECTOR_SPAN_MIN = 45
DETECTOR_SPAN_MAX = 270
DETECTOR_SPAN_STEP = 45
DETECTOR_SPAN_DEFAULT = 180
DETECTOR_SPAN_SECOND_EXPERIMENT = 270

MAIN_FILE = 'imgs/Shepp_logan.jpg'
FILES = ['imgs/Shepp_logan.jpg', 'imgs/CT_ScoutView.jpg']

MASK_SIZE = 11

# plot config
X_LABEL = {
    "detector_count": "Liczba detektorów",
    "iteration_count": "Liczba iteracji",
    "detector_span": "Rozpiętość wachlarza"
}
Y_LABEL = "RMSE"
TITLE = {
    "detector_count": "Wykres wartości RMSE od liczby detektorów",
    "iteration_count": "Wykres wartości RMSE od liczby iteracji",
    "detector_span": "Wykres wartości RMSE od rozpiętości wachlarza"
}

### Functions

In [ ]:
def load_image(filename):
    return transform.normalize_img(imread(filename, as_gray=True))
    
    
def experiment_input(filename):
    input_image = load_image(filename)
    show_input_image(input_image)
    image, offset = transform.resize_to_square(input_image)
    r = transform.calc_radius(image)
    return input_image, image, offset, r


def display_image(image):
    with BytesIO() as file:
        PIL.Image.fromarray(np.uint8(image * 255)).save(file, format='png')
        display(Image(data=file.getvalue(), width="30%"))
        

def show_input_image(image):
    display(Markdown("### Input image"))
    display_image(image)
    

def display_experiment_header(detector_count, iteration_count, detector_span):
    display(Markdown(
        "*** \n"
        f"**Detector count**: {detector_count}<br>"
        f"**Iteration count**: {iteration_count}<br>"
        f"**Detector span**: {detector_span}"))
    

def main_experiment_step(input_image, image, offset, r, detector_count = None, iteration_count = None, detector_span = None):
    # setup params
    if detector_count is None:
        detector_count = DETECTOR_COUNT_DEFAULT
    if iteration_count is None:
        iteration_count = ITERATION_COUNT_DEFAULT
    if detector_span is None:
        detector_span = DETECTOR_SPAN_DEFAULT
    
    display_experiment_header(detector_count, iteration_count, detector_span)
    
    emitter_step_rad = np.pi / iteration_count
    detector_span_rad = np.deg2rad(detector_span)
    
    sinogram = transform.img_to_sinogram(image, emitter_step_rad, r, detector_count, detector_span_rad)
    filtered_sinogram = transform.convolve(sinogram, MASK_SIZE)
    
    filtered_output_image = transform.sinogram_to_img_simple(filtered_sinogram, emitter_step_rad, r, detector_count, detector_span_rad, offset)
    output = filtered_output_image
    rmse = transform.root_mean_square_error(input_image, output)
    
    display(Markdown("##### Output: "))
    display_image(output)
    display(Markdown(f"**RMSE**: {rmse}"))
    
    # returns rmse of output, filtered output and filtered output after gaussian
    return rmse


def main_experiment(input_image, image, offset, r, property_name, min_value, max_value, step):
    title = " ".join(property_name.split("_")).capitalize()
    display(Markdown(f"### {title}"))
    display(Markdown(f"##### From {min_value} to {max_value} with step {step}"))
    x = []
    y = []
    for value in range(min_value, max_value + step, step):
        rmse = main_experiment_step(input_image, image, offset, r, **{property_name: value})
        x.append(value)
        y.append(rmse)
    y = np.array(y).T
    plt.title(TITLE[property_name])
    plt.xlabel(X_LABEL[property_name])
    plt.ylabel(Y_LABEL)
    plt.plot(x, y)
    plt.show()
    
    
def second_experiment(filename, detector_count = None, iteration_count = None, detector_span = None):
    input_image, image, offset, r = experiment_input(filename)
    # setup params
    if detector_count is None:
        detector_count = DETECTOR_COUNT_SECOND_EXPERIMENT
    if iteration_count is None:
        iteration_count = ITERATION_COUNT_SECOND_EXPERIMENT
    if detector_span is None:
        detector_span = DETECTOR_SPAN_SECOND_EXPERIMENT
        
    display_experiment_header(detector_count, iteration_count, detector_span)
    
    emitter_step_rad = np.pi / iteration_count
    detector_span_rad = np.deg2rad(detector_span)
    
    emitter_step_rad = np.pi / iteration_count
    detector_span_rad = np.deg2rad(detector_span)
    
    sinogram = transform.img_to_sinogram(image, emitter_step_rad, r, detector_count, detector_span_rad)
    filtered_sinogram = transform.convolve(sinogram, MASK_SIZE)
    
    output_image = transform.sinogram_to_img_simple(sinogram, emitter_step_rad, r, detector_count, detector_span_rad, offset)
    filtered_output_image = transform.sinogram_to_img_simple(filtered_sinogram, emitter_step_rad, r, detector_count, detector_span_rad, offset)
    gaussian_output_image = transform.apply_gaussian(filtered_output_image)
    
    display(Markdown("##### Output: "))
    display_image(output_image)
    display(Markdown(f"RMSE: {transform.root_mean_square_error(input_image, output_image)}"))
    display(Markdown("##### Convolution: "))
    display_image(filtered_output_image)
    display(Markdown(f"RMSE: {transform.root_mean_square_error(input_image, filtered_output_image)}"))
    display(Markdown("##### Convolution + gaussian: "))
    display_image(gaussian_output_image)
    display(Markdown(f"**RMSE**: {transform.root_mean_square_error(input_image, gaussian_output_image)}"))
    


### Main

In [ ]:
# A, B
display(Markdown("# Main experiment"))

data = experiment_input(MAIN_FILE)
main_experiment(*data, "detector_count", DETECTOR_COUNT_MIN, DETECTOR_COUNT_MAX, DETECTOR_COUNT_STEP)
main_experiment(*data, "iteration_count", ITERATION_COUNT_MIN, ITERATION_COUNT_MAX, ITERATION_COUNT_STEP)
main_experiment(*data, "detector_span", DETECTOR_SPAN_MIN, DETECTOR_SPAN_MAX, DETECTOR_SPAN_STEP)

# C
display(Markdown("# Second experiment"))
for filename in FILES:
    result = second_experiment(filename)

display(Markdown("# DONE!!!"))